- KRX 에서 종가, 거래량, PER, PBR, 투자자별 거래실적 가져오기
- 어제 데이터 가져오기 

In [1]:
#!pip install pykrx
from pykrx import stock

In [2]:
stocknames= []
codes = ['005930', '000660', '051910', '066570', '011070', '018260', '009150', '032830',
             '000810', '017670', '030200', '000720', '028050', '003490', '005380', '000270', 
             '271560', '097950', '007310', '006800', '071050', '005940', '051900', '090430', 
             '002790', '035250', '008770', '105560', '055550', '086790', '023530', '139480', 
             '004170', '007070', '035420', '035720', '035760', '253450', '207940', '068270',
             '128940', '036570', '251270', '009830', '006260', '005490', '010130', '010950', 
             '096770', '011200']
for code in codes:
    stockname = stock.get_market_ticker_name(code)
    stocknames.append(stockname)
print(len(stocknames), stocknames)


50 ['삼성전자', 'SK하이닉스', 'LG화학', 'LG전자', 'LG이노텍', '삼성에스디에스', '삼성전기', '삼성생명', '삼성화재', 'SK텔레콤', 'KT', '현대건설', '삼성엔지니어링', '대한항공', '현대차', '기아', '오리온', 'CJ제일제당', '오뚜기', '미래에셋증권', '한국금융지주', 'NH투자증권', 'LG생활건강', '아모레퍼시픽', '아모레G', '강원랜드', '호텔신라', 'KB금융', '신한지주', '하나금융지주', '롯데쇼핑', '이마트', '신세계', 'GS리테일', 'NAVER', '카카오', 'CJ ENM', '스튜디오드래곤', '삼성바이오로직스', '셀트리온', '한미약품', '엔씨소프트', '넷마블', '한화솔루션', 'LS', 'POSCO', '고려아연', 'S-Oil', 'SK이노베이션', 'HMM']


In [3]:
# 날짜
from datetime import date, timedelta

today = date.today().strftime('%Y%m%d')
yesterday = (date.today() - timedelta(1)).strftime('%Y%m%d')
bfyday = (date.today() - timedelta(2)).strftime('%Y%m%d')
bffyday = (date.today() - timedelta(3)).strftime('%Y%m%d')

In [21]:
yesterday 

'20211006'

# 거래량

In [4]:
# 개별종목 시세 추이
df = stock.get_market_ohlcv_by_date(bffyday,yesterday, ticker="005930")
df

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-10-05,73000,73000,71400,72200,24013921
2021-10-06,72600,72800,71200,71300,18956962


#### atr  : 전일종가,고가,저가 중 차이가 큰 값을 구하기

In [5]:
df_atr = df.rename(index={'날짜':'Date'}).drop(columns="시가", axis=1)
df_atr.columns= ['high', 'low','close','volume']
df_atr["atr"] = ""
df_atr

,high,low,close,volume,atr
날짜,,,,,
2021-10-05,73000,71400,72200,24013921,
2021-10-06,72800,71200,71300,18956962,


In [6]:
lst = []
atr = []
# 0->날짜, 1->종가, 2->고가, 3->저가
# 0->고가, 1->저가, 2->종가
for i in range(len(df_atr)-1):
    a = df_atr.iloc[i,0] - df_atr.iloc[i,1]  # 고가-저가
    b = df_atr.iloc[i,0] - df_atr.iloc[i+1,2]  # 고가-전날종가
    c = df_atr.iloc[i,1] - df_atr.iloc[i+1,2]  # 저가-전날종가
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))
df_atr.iloc[1,4] = atr
df_atr

,high,low,close,volume,atr
날짜,,,,,
2021-10-05,73000,71400,72200,24013921,
2021-10-06,72800,71200,71300,18956962,1700


In [7]:
type(max(lst)), type(atr)

(numpy.int32, list)

In [8]:
df_atr = df_atr[1:]
print(df_atr[['close','volume','atr']])

            close    volume   atr
날짜                               
2021-10-06  71300  18956962  1700


# PER, PBR

In [9]:
# PER, PBR

df_per = stock.get_market_fundamental_by_date(yesterday, yesterday, "005930")  # 어제 데이터 가져오기
df_per

,BPS,PER,PBR,EPS,DIV,DPS
날짜,,,,,,
2021-10-06,39406,18.56,1.81,3841,4.2,2994


In [10]:
df_per = df_per[['PER', 'PBR']]
df_per

,PER,PBR
날짜,,
2021-10-06,18.56,1.81


# 거래실적  
#### 업데이트 늦음

In [11]:
# 투자자별 거래실적

df_value = stock.get_market_trading_value_by_date(yesterday, yesterday, "005930")
df_value

,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,
2021-10-06,-25002783100,2975716600,98910041100,-76882974600,0


In [13]:
df_value.columns

Index(['기관합계', '기타법인', '개인', '외국인합계', '전체'], dtype='object')

In [14]:
df_value= df_value.drop(['전체'], axis = 1)
df_value

,기관합계,기타법인,개인,외국인합계
날짜,,,,
2021-10-06,-25002783100,2975716600,98910041100,-76882974600


In [12]:
df_v = stock.get_market_trading_value_by_date(today, today, "005930")  # 오늘 데이터
df_v

,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,
2021-10-07,99892861200,-180317200,90663441900,-190375985900,0


# 컬럼 수정

In [15]:
import pandas as pd

df = pd.concat([df_atr, df_per, df_value],axis=1)
df

,high,low,close,volume,atr,PER,PBR,기관합계,기타법인,개인,외국인합계
날짜,,,,,,,,,,,
2021-10-06,72800,71200,71300,18956962,1700,18.56,1.81,-25002783100,2975716600,98910041100,-76882974600


In [16]:
df = df.reset_index()
df_new = df.rename(columns={'날짜': 'Date'})
df_new

,Date,high,low,close,volume,atr,PER,PBR,기관합계,기타법인,개인,외국인합계
0,2021-10-06,72800,71200,71300,18956962,1700,18.56,1.81,-25002783100,2975716600,98910041100,-76882974600


# 함수화

In [35]:
from pykrx import stock
import pandas as pd

class UpdateDB:
    def __init__(self):
        self.yday = (date.today() - timedelta(1)).strftime('%Y%m%d')
        self.bfyday = (date.today() - timedelta(2)).strftime('%Y%m%d')
        self.bffyday = (date.today() - timedelta(3)).strftime('%Y%m%d')

    def getKRX(self, code):
        self.stockname = stock.get_market_ticker_name(code)
        def getATR():
            df_a = stock.get_market_ohlcv_by_date(self.bffyday, self.yday, code)
            df_atr = df_a.rename(index={'날짜': 'Date'}).drop(columns="시가", axis=1)
            df_atr.columns = ['high', 'low', 'close', 'volume']
            df_atr["atr"] = ""
            lst = []
            atr = []
            for i in range(len(df_atr) - 1):
                a = df_atr.iloc[i, 0] - df_atr.iloc[i, 1]  # 고가-저가
                b = df_atr.iloc[i, 0] - df_atr.iloc[i + 1, 2]  # 고가-전날종가
                c = df_atr.iloc[i, 1] - df_atr.iloc[i + 1, 2]  # 저가-전날종가
                lst = [abs(a), abs(b), abs(c)]
                atr.append(max(lst))
            df_atr.iloc[1, 4] = atr
            df_atr = df_atr[1:]
            return df_atr[['close','volume','atr']]
        self.ohlcv = getATR()
        self.funda = stock.get_market_fundamental_by_date(self.yday, self.yday, code)[['PER', 'PBR']]
        self.value = stock.get_market_trading_value_by_date(self.yday, self.yday, code).drop(['전체'], axis=1)

        self.df = pd.concat([self.ohlcv, self.funda, self.value], axis=1)
        self.df_krx = self.df.reset_index()
        self.df_krx.columns=['date','y','volume', 'atr', 'per', 'pbr', 'institution', 'corp', 'retail', 'foreign']
        self.df_krx = self.df_krx.set_index('date')
        return self.df_krx


if __name__ == "__main__":

    updatedb = UpdateDB()
    updatedb.getKRX('005930')

                y    volume   atr    per   pbr  institution        corp  \
date                                                                      
2021-10-06  71300  18956962  1700  18.56  1.81 -25002783100  2975716600   

                 retail      foreign  
date                                  
2021-10-06  98910041100 -76882974600  
